## Topic: Apriori algorithm, generate and print out all the association rules.

#### Name: Veena Chaudhari

#### Subject: CS 634 Data mining


## Loading Libraries

In [1]:
#importing all libraries
import pandas as pd
import numpy as np
import csv
from itertools import combinations

## Defining functions

In [2]:
# For loading data and creating a list of items
def data_prep(file):
    df = pd.read_csv(file).iloc[:,1:]
    data1 = dict()
    for i, row in df.iterrows():
        data1[i] = list(row)
        
    listed = list(data1.values())
    unique_list = np.unique(np.concatenate(listed))[np.unique(np.concatenate(listed)) != 'nan']
    oneitem_list = {key: None for key in unique_list}
    return oneitem_list, data1
    

In [3]:
# Create itemset for specified number of combinations
def create_itemset(k,itemset):
    unique_itemset = np.unique(list(itemset.keys()))
    items_list = list(combinations(unique_itemset, k))
    return items_list

In [4]:
#Calculate the frequency of each itemset (1, 2, 3, itemsets) by checking the number of transactions in the database
def frequency (item_list, data):
    itemset_count = dict()
    for i in range(len(item_list)):
        counter = 0
        for j in range(len(data)):
            if all(elem in data[j] for elem in item_list[i]):
                counter += 1
        itemset_count[item_list[i]] = counter
    return itemset_count

In [5]:
# Check if itemset are superset of previously removed transactionc
def check_if_superset(itemset, previ, n):
    if n>2:    
        subsets = list(combinations(itemset, (n-1)))
        if any(elem in previ for elem in subsets):
            return False
    return True

In [6]:
# Check if the itemset satisfy specified minimum support
def min_support(item_freq, data,min_sup, n,previous_removed):
    min_sup = min_sup/100
    length = len(data)
    return_item = {}
    removed_item = []
    for key, value in item_freq.items():
        if value/length < min_sup:
            removed_item.append(key)
        else:
             if check_if_superset(key, previous_removed, n) == True :
                    return_item[key] = value
             
    return return_item, removed_item

In [7]:
# Main Function to calculate all item sets and print rules
def main(file,minimum_support, minimum_confidence):
    min_confidence = minimum_confidence/100 
    itemlist, data = data_prep(file)
    
    # 1-item_set
    prev = []
    one_item_min ={}
    one_item_list = create_itemset(1,itemlist)
    one_item_freq = frequency(one_item_list, data)
    one_min, removed_item_1 = min_support(one_item_freq, data, minimum_support, 1,prev)
    for keys, values in one_min.items():
        one_item_min[keys[0]] = values

    # 2 item-set
    two_item_list = create_itemset(2,one_item_min)
    two_item_freq = frequency(two_item_list, data)
    two_item_min, removed_item_2 = min_support(two_item_freq, data, minimum_support, 2, removed_item_1)

    # 3 item-set
    three_item_list = create_itemset(3,two_item_min)
    three_item_freq = frequency(three_item_list, data)
    three_item_min, removed_item_3 = min_support(three_item_freq, data, minimum_support, 3,removed_item_2)

    #four item-set
    four_item_list = create_itemset(4,three_item_min)
    four_item_freq = frequency(four_item_list, data)
    four_item_min, removed_item_4 = min_support(four_item_freq, data, minimum_support, 4,removed_item_3)

    itemset_all = {**one_item_min, **two_item_min, **three_item_min, **four_item_min} 
    
# For printing all association rules for two itemset
    printing = pd.DataFrame()
    for i in two_item_min:
            confidence_1 = np.round (itemset_all[i]/itemset_all[i[0]], decimals = 2)
            confidence_2 = np.round (itemset_all[i]/itemset_all[i[1]], decimals = 2)
            if confidence_1 > min_confidence:
                new_row_2_1 = {'Antecedent' : i[0],'Consequent' : i[1],'Rule_Confidence' : confidence_1 }
                printing = printing.append(new_row_2_1, ignore_index=True)
            else: 
                pass
            if confidence_2 > min_confidence:
                new_row_2_2 = {'Antecedent' : i[1],'Consequent' : i[0],'Rule_Confidence' : confidence_2 }
                printing = printing.append(new_row_2_2, ignore_index=True)
            else: 
                pass
#printing all association rules for three itemset
    for i in three_item_min:
        combos = list(combinations(i, 2))
        for j in combos:
            confidence = np.round(itemset_all[i]/itemset_all[j], decimals = 2)
            if confidence > min_confidence:
                trans2 = list(set(i) - set(j))
                new_row_3 = {'Antecedent' : j,'Consequent':trans2,'Rule_Confidence' : confidence }
                printing = printing.append(new_row_3, ignore_index=True)
            else: 
                pass
#printing all association rules for four itemset            
    for i in four_item_min:
        combos = list(combinations(i, 3))
        for j in combos:
            confidence = np.round(itemset_all[i]/itemset_all[j], decimals = 2)
            if confidence > min_confidence:
                trans2 = list(set(i) - set(j))
                new_row_4 = {'Antecedent' : j,'Consequent':trans2,'Rule_Confidence' : confidence }
                printing = printing.append(new_row_4, ignore_index=True)
            else: 
                pass    
    
    return itemset_all,printing

## Determining Association Rules for K-Mart

#### Running Kmart dataset with support 40% and confidence 50%

In [8]:
#File for dataset
kmart = '/Users/veena/Desktop/Dmining/Projects/Datasets/K-mart.csv'
#Specify minimum support
minimum_support = 40
#Specify minimum confidence
minimum_confidence = 50

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_kmart_1 = main(kmart,minimum_support,minimum_confidence)

print('***************** KMART *****************')
#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Kmart with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_kmart_1,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Kmart with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_kmart_1['Rule_Confidence'])):
    print ("{}) {} --> {} ".format(i+1, rules_kmart_1['Antecedent'][i],rules_kmart_1['Consequent'][i]))

***************** KMART *****************

 All association rules for Kmart with support = 40 and Confidence = 50 

                    Antecedent      Consequent  Rule_Confidence
0                   Bed Skirts    Kids Bedding             0.91
1                 Kids Bedding      Bed Skirts             0.83
2                   Bed Skirts           Shams             0.82
3                        Shams      Bed Skirts             0.82
4                   Bed Skirts          Sheets             0.82
5                       Sheets      Bed Skirts             0.90
6                 Kids Bedding           Shams             0.75
7                        Shams    Kids Bedding             0.82
8                 Kids Bedding          Sheets             0.83
9                       Sheets    Kids Bedding             1.00
10  (Bed Skirts, Kids Bedding)         [Shams]             0.80
11         (Bed Skirts, Shams)  [Kids Bedding]             0.89
12       (Kids Bedding, Shams)    [Bed Skirts]      

#### Running Kmart dataset with support 50% and confidence 75%

In [9]:
#Specify minimum support
minimum_support = 50
#Specify minimum confidence
minimum_confidence = 75

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_kmart = main(kmart,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Kmart with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_kmart,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Kmart with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_kmart['Rule_Confidence'])):
    print ("{}) {} --> {} ".format(i+1, rules_kmart['Antecedent'][i],rules_kmart['Consequent'][i]))


 All association rules for Kmart with support = 50 and Confidence = 75 

     Antecedent    Consequent  Rule_Confidence
0    Bed Skirts  Kids Bedding             0.91
1  Kids Bedding    Bed Skirts             0.83
2  Kids Bedding        Sheets             0.83
3        Sheets  Kids Bedding             1.00 

---------------------------------------------------------------------

Final Association Rules for Kmart with support = 50 and Confidence = 75 

1) Bed Skirts --> Kids Bedding 
2) Kids Bedding --> Bed Skirts 
3) Kids Bedding --> Sheets 
4) Sheets --> Kids Bedding 


#### Running Kmart dataset with support 30% and confidence 90%

In [10]:
#Specify minimum support
minimum_support = 30
#Specify minimum confidence
minimum_confidence = 90

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_kmart = main(kmart,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Kmart with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_kmart,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Kmart with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_kmart['Rule_Confidence'])):
    print ("{}) {} --> {} ".format(i+1, rules_kmart['Antecedent'][i],rules_kmart['Consequent'][i]))


 All association rules for Kmart with support = 30 and Confidence = 90 

                                Antecedent      Consequent  Rule_Confidence
0                               Bedspreads      Bed Skirts             1.00
1                               Bed Skirts    Kids Bedding             0.91
2                               Bedspreads    Kids Bedding             1.00
3                               Bedspreads          Sheets             1.00
4                                   Sheets    Kids Bedding             1.00
5                 (Bed Skirts, Bedspreads)  [Kids Bedding]             1.00
6               (Bedspreads, Kids Bedding)    [Bed Skirts]             1.00
7                 (Bed Skirts, Bedspreads)        [Sheets]             1.00
8                     (Bedspreads, Sheets)    [Bed Skirts]             1.00
9                     (Bed Skirts, Sheets)  [Kids Bedding]             1.00
10                         (Shams, Sheets)    [Bed Skirts]             1.00
11            

#### Running Kmart dataset with support 35% and confidence 60%

In [11]:
#Specify minimum support
minimum_support = 35
#Specify minimum confidence
minimum_confidence = 60

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_kmart = main(kmart,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Kmart with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_kmart,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Kmart with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_kmart['Rule_Confidence'])):
    print ("{}) {} --> {} ".format(i+1, rules_kmart['Antecedent'][i],rules_kmart['Consequent'][i]))


 All association rules for Kmart with support = 35 and Confidence = 60 

                                Antecedent      Consequent  Rule_Confidence
0                               Bed Skirts      Bedspreads             0.64
1                               Bedspreads      Bed Skirts             1.00
2                               Bed Skirts    Kids Bedding             0.91
3                             Kids Bedding      Bed Skirts             0.83
4                               Bed Skirts           Shams             0.82
5                                    Shams      Bed Skirts             0.82
6                               Bed Skirts          Sheets             0.82
7                                   Sheets      Bed Skirts             0.90
8                               Bedspreads    Kids Bedding             1.00
9                               Bedspreads          Sheets             1.00
10                                  Sheets      Bedspreads             0.70
11            

## Determining Association Rules for Best_buy

#### Running Best Buy dataset with support 40% and confidence 90%

In [12]:
bb = '/Users/veena/Desktop/Dmining/Projects/Datasets/Best_buy.csv'
#Specify minimum support
minimum_support = 40
#Specify minimum confidence
minimum_confidence = 90


print('***************** BEST BUY *****************')
#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_bb = main(bb,minimum_support,minimum_confidence)
print()
#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Best buy with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_bb,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Best buy with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_bb['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_bb['Antecedent'][i],rules_bb['Consequent'][i]))


***************** BEST BUY *****************


 All association rules for Best buy with support = 40 and Confidence = 90 

                            Antecedent     Consequent  Rule_Confidence
0                  External Hard-Drive     Anti-Virus              1.0
1                     Microsoft Office    Flash Drive              1.0
2                              Printer    Flash Drive              1.0
3  (External Hard-Drive, Lab Top Case)   [Anti-Virus]              1.0
4          (Flash Drive, Lab Top Case)   [Anti-Virus]              1.0
5       (Anti-Virus, Microsoft Office)  [Flash Drive]              1.0
6          (Microsoft Office, Printer)  [Flash Drive]              1.0 

---------------------------------------------------------------------
Final Association Rules for Best buy with support = 40 and Confidence = 90 

1) External Hard-Drive --> Anti-Virus  
2) Microsoft Office --> Flash Drive  
3) Printer --> Flash Drive  
4) ('External Hard-Drive', 'Lab Top Case') --> ['Anti

#### Running Best Buy dataset with support 30% and confidence 80%

In [13]:
#Specify minimum support
minimum_support = 55
#Specify minimum confidence
minimum_confidence = 80

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_bb = main(bb,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Best buy with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_bb,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Best buy with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_bb['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_bb['Antecedent'][i],rules_bb['Consequent'][i]))



 All association rules for Best buy with support = 55 and Confidence = 80 

         Antecedent        Consequent  Rule_Confidence
0        Anti-Virus      Lab Top Case             0.86
1      Lab Top Case        Anti-Virus             0.86
2       Flash Drive  Microsoft Office             0.85
3  Microsoft Office       Flash Drive             1.00 

---------------------------------------------------------------------
Final Association Rules for Best buy with support = 55 and Confidence = 80 

1) Anti-Virus --> Lab Top Case  
2) Lab Top Case --> Anti-Virus  
3) Flash Drive --> Microsoft Office  
4) Microsoft Office --> Flash Drive  


#### Running Best Buy dataset with support 45% and confidence 95%

In [14]:
#Specify minimum support
minimum_support = 45
#Specify minimum confidence
minimum_confidence = 95

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_bb = main(bb,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Best buy with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_bb,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Best buy with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_bb['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_bb['Antecedent'][i],rules_bb['Consequent'][i]))



 All association rules for Best buy with support = 45 and Confidence = 95 

                    Antecedent     Consequent  Rule_Confidence
0          External Hard-Drive     Anti-Virus              1.0
1             Microsoft Office    Flash Drive              1.0
2                      Printer    Flash Drive              1.0
3  (Flash Drive, Lab Top Case)   [Anti-Virus]              1.0
4  (Microsoft Office, Printer)  [Flash Drive]              1.0 

---------------------------------------------------------------------
Final Association Rules for Best buy with support = 45 and Confidence = 95 

1) External Hard-Drive --> Anti-Virus  
2) Microsoft Office --> Flash Drive  
3) Printer --> Flash Drive  
4) ('Flash Drive', 'Lab Top Case') --> ['Anti-Virus']  
5) ('Microsoft Office', 'Printer') --> ['Flash Drive']  


#### Running Best Buy dataset with support 35% and confidence 90%

In [15]:
#Specify minimum support
minimum_support = 35
#Specify minimum confidence
minimum_confidence = 90

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_bb = main(bb,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Best buy with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_bb,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Best buy with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_bb['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_bb['Antecedent'][i],rules_bb['Consequent'][i]))



 All association rules for Best buy with support = 35 and Confidence = 90 

                                       Antecedent     Consequent  \
0                             External Hard-Drive     Anti-Virus   
1                                Microsoft Office    Flash Drive   
2                                         Printer    Flash Drive   
3             (External Hard-Drive, Lab Top Case)   [Anti-Virus]   
4                     (Flash Drive, Lab Top Case)   [Anti-Virus]   
5                  (Anti-Virus, Microsoft Office)  [Flash Drive]   
6                           (Anti-Virus, Printer)  [Flash Drive]   
7                (Lab Top Case, Microsoft Office)   [Anti-Virus]   
8                (Lab Top Case, Microsoft Office)  [Flash Drive]   
9                     (Microsoft Office, Printer)  [Flash Drive]   
10   (Anti-Virus, Lab Top Case, Microsoft Office)  [Flash Drive]   
11  (Flash Drive, Lab Top Case, Microsoft Office)   [Anti-Virus]   

    Rule_Confidence  
0               

## Determining Association Rules for Costco

#### Running Costco dataset with support 40% and confidence 30%

In [16]:
costco = '/Users/veena/Desktop/Dmining/Projects/Datasets/costco.csv'
#Specify minimum support
minimum_support = 40
#Specify minimum confidence
minimum_confidence = 30

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_costco = main(costco,minimum_support,minimum_confidence)

print('***************** COSTCO *****************')
#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Costco with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_costco,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for costco with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_costco['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_costco['Antecedent'][i],rules_costco['Consequent'][i]))

***************** COSTCO *****************

 All association rules for Costco with support = 40 and Confidence = 30 

                              Antecedent           Consequent  Rule_Confidence
0                          Peanut Butter      Roasted seaweed             1.00
1                        Roasted seaweed        Peanut Butter             0.77
2                          Peanut Butter    chocolate almonds             0.90
3                      chocolate almonds        Peanut Butter             0.82
4                        Roasted seaweed    chocolate almonds             0.69
5                      chocolate almonds      Roasted seaweed             0.82
6                        Roasted seaweed               coffee             0.69
7                                 coffee      Roasted seaweed             0.69
8       (Peanut Butter, Roasted seaweed)  [chocolate almonds]             0.90
9     (Peanut Butter, chocolate almonds)    [Roasted seaweed]             1.00
10  (Roasted 

#### Running Costco dataset with support 45% and confidence 65%

In [17]:
#Specify minimum support
minimum_support = 45
#Specify minimum confidence
minimum_confidence = 65

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_costco = main(costco,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Costco with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_costco,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for costco with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_costco['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_costco['Antecedent'][i],rules_costco['Consequent'][i]))


 All association rules for Costco with support = 45 and Confidence = 65 

                              Antecedent           Consequent  Rule_Confidence
0                          Peanut Butter      Roasted seaweed             1.00
1                        Roasted seaweed        Peanut Butter             0.77
2                          Peanut Butter    chocolate almonds             0.90
3                      chocolate almonds        Peanut Butter             0.82
4                        Roasted seaweed    chocolate almonds             0.69
5                      chocolate almonds      Roasted seaweed             0.82
6                        Roasted seaweed               coffee             0.69
7                                 coffee      Roasted seaweed             0.69
8       (Peanut Butter, Roasted seaweed)  [chocolate almonds]             0.90
9     (Peanut Butter, chocolate almonds)    [Roasted seaweed]             1.00
10  (Roasted seaweed, chocolate almonds)      [Peanut Bu

#### Running Costco dataset with support 30% and confidence 75%

In [18]:
#Specify minimum support
minimum_support = 30
#Specify minimum confidence
minimum_confidence = 75

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_costco = main(costco,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Costco with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_costco,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for costco with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_costco['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_costco['Antecedent'][i],rules_costco['Consequent'][i]))


 All association rules for Costco with support = 30 and Confidence = 75 

                             Antecedent           Consequent  Rule_Confidence
0                         Peanut Butter      Roasted seaweed             1.00
1                       Roasted seaweed        Peanut Butter             0.77
2                         Peanut Butter    chocolate almonds             0.90
3                     chocolate almonds        Peanut Butter             0.82
4                     chocolate almonds      Roasted seaweed             0.82
5      (Peanut Butter, Roasted seaweed)  [chocolate almonds]             0.90
6    (Peanut Butter, chocolate almonds)    [Roasted seaweed]             1.00
7  (Roasted seaweed, chocolate almonds)      [Peanut Butter]             1.00
8               (Peanut Butter, coffee)    [Roasted seaweed]             1.00 

---------------------------------------------------------------------
Final Association Rules for costco with support = 30 and Confidence = 75 

#### Running Costco dataset with support 40% and confidence 40%

In [19]:
#Specify minimum support
minimum_support = 40
#Specify minimum confidence
minimum_confidence = 40

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_costco = main(costco,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Costco with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_costco,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for costco with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_costco['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_costco['Antecedent'][i],rules_costco['Consequent'][i]))


 All association rules for Costco with support = 40 and Confidence = 40 

                              Antecedent           Consequent  Rule_Confidence
0                          Peanut Butter      Roasted seaweed             1.00
1                        Roasted seaweed        Peanut Butter             0.77
2                          Peanut Butter    chocolate almonds             0.90
3                      chocolate almonds        Peanut Butter             0.82
4                        Roasted seaweed    chocolate almonds             0.69
5                      chocolate almonds      Roasted seaweed             0.82
6                        Roasted seaweed               coffee             0.69
7                                 coffee      Roasted seaweed             0.69
8       (Peanut Butter, Roasted seaweed)  [chocolate almonds]             0.90
9     (Peanut Butter, chocolate almonds)    [Roasted seaweed]             1.00
10  (Roasted seaweed, chocolate almonds)      [Peanut Bu

## Determining Association Rules for Nike

#### Running Nike dataset with support 60% and confidence 60%

In [20]:
nike = '/Users/veena/Desktop/Dmining/Projects/Datasets/Nike.csv'
#Specify minimum support
minimum_support = 60
#Specify minimum confidence
minimum_confidence = 60
#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_nike = main(nike,minimum_support,minimum_confidence)

print('***************** NIKE *****************')
#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Nike with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_nike,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Nike with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_nike['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_nike['Antecedent'][i],rules_nike['Consequent'][i]))

***************** NIKE *****************

 All association rules for Nike with support = 60 and Confidence = 60 

    Antecedent   Consequent  Rule_Confidence
0        Socks  Sweatshirts             0.92
1  Sweatshirts        Socks             0.92 

---------------------------------------------------------------------
Final Association Rules for Nike with support = 60 and Confidence = 60 

1) Socks --> Sweatshirts  
2) Sweatshirts --> Socks  


#### Running Nike dataset with support 60% and confidence 90%

In [21]:
#Specify minimum support
minimum_support = 60
#Specify minimum confidence
minimum_confidence = 90

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_nike = main(nike,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Nike with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_nike,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Nike with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_nike['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_nike['Antecedent'][i],rules_nike['Consequent'][i]))


 All association rules for Nike with support = 60 and Confidence = 90 

    Antecedent   Consequent  Rule_Confidence
0        Socks  Sweatshirts             0.92
1  Sweatshirts        Socks             0.92 

---------------------------------------------------------------------
Final Association Rules for Nike with support = 60 and Confidence = 90 

1) Socks --> Sweatshirts  
2) Sweatshirts --> Socks  


#### Running Nike dataset with support 55% and confidence 55%

In [22]:
#Specify minimum support
minimum_support = 55
#Specify minimum confidence
minimum_confidence = 55

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_nike = main(nike,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Nike with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_nike,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Nike with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_nike['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_nike['Antecedent'][i],rules_nike['Consequent'][i]))


 All association rules for Nike with support = 55 and Confidence = 55 

     Antecedent    Consequent  Rule_Confidence
0  Running Shoe         Socks             0.79
1         Socks  Running Shoe             0.85
2  Running Shoe   Sweatshirts             0.79
3   Sweatshirts  Running Shoe             0.85
4         Socks   Sweatshirts             0.92
5   Sweatshirts         Socks             0.92 

---------------------------------------------------------------------
Final Association Rules for Nike with support = 55 and Confidence = 55 

1) Running Shoe --> Socks  
2) Socks --> Running Shoe  
3) Running Shoe --> Sweatshirts  
4) Sweatshirts --> Running Shoe  
5) Socks --> Sweatshirts  
6) Sweatshirts --> Socks  


#### Running Nike dataset with support 55% and confidence 70%

In [23]:
#Specify minimum support
minimum_support = 55
#Specify minimum confidence
minimum_confidence = 70

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_nike = main(nike,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Nike with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_nike,'\n')
print('---------------------------------------------------------------------')
#Printing Final association rules in format A,B --> C
print("Final Association Rules for Nike with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_nike['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_nike['Antecedent'][i],rules_nike['Consequent'][i]))


 All association rules for Nike with support = 55 and Confidence = 70 

     Antecedent    Consequent  Rule_Confidence
0  Running Shoe         Socks             0.79
1         Socks  Running Shoe             0.85
2  Running Shoe   Sweatshirts             0.79
3   Sweatshirts  Running Shoe             0.85
4         Socks   Sweatshirts             0.92
5   Sweatshirts         Socks             0.92 

---------------------------------------------------------------------
Final Association Rules for Nike with support = 55 and Confidence = 70 

1) Running Shoe --> Socks  
2) Socks --> Running Shoe  
3) Running Shoe --> Sweatshirts  
4) Sweatshirts --> Running Shoe  
5) Socks --> Sweatshirts  
6) Sweatshirts --> Socks  


## Determining Association Rules for Generic

#### Running Generic dataset with support 20% and confidence 90%

In [24]:
generic = '/Users/veena/Desktop/Dmining/Projects/Datasets/Generic.csv'
#Specify minimum support
minimum_support = 20
#Specify minimum confidence
minimum_confidence = 90

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_generic = main(generic,minimum_support,minimum_confidence)

print('***************** GENERIC *****************')
#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Generic with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_generic,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules Generic with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_generic['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_generic['Antecedent'][i],rules_generic['Consequent'][i]))

***************** GENERIC *****************

 All association rules for Generic with support = 20 and Confidence = 90 

   Antecedent Consequent  Rule_Confidence
0           B          A              1.0
1           C          A              1.0
2           D          A              1.0
3           E          A              1.0
4      (B, C)        [A]              1.0
5      (B, D)        [A]              1.0
6      (B, E)        [A]              1.0
7      (C, D)        [A]              1.0
8      (C, E)        [A]              1.0
9      (D, E)        [A]              1.0
10  (B, C, D)        [A]              1.0 

---------------------------------------------------------------------

Final Association Rules Generic with support = 20 and Confidence = 90 

1) B --> A  
2) C --> A  
3) D --> A  
4) E --> A  
5) ('B', 'C') --> ['A']  
6) ('B', 'D') --> ['A']  
7) ('B', 'E') --> ['A']  
8) ('C', 'D') --> ['A']  
9) ('C', 'E') --> ['A']  
10) ('D', 'E') --> ['A']  
11) ('B', 'C', 'D') --

#### Running Generic dataset with support 35% and confidence 95%

In [25]:
#Specify minimum support
minimum_support = 35
#Specify minimum confidence
minimum_confidence = 95

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_generic = main(generic,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Generic with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_generic,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules Generic with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_generic['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_generic['Antecedent'][i],rules_generic['Consequent'][i]))


 All association rules for Generic with support = 35 and Confidence = 95 

  Antecedent Consequent  Rule_Confidence
0          B          A              1.0
1          C          A              1.0
2          D          A              1.0
3          E          A              1.0
4     (B, C)        [A]              1.0 

---------------------------------------------------------------------

Final Association Rules Generic with support = 35 and Confidence = 95 

1) B --> A  
2) C --> A  
3) D --> A  
4) E --> A  
5) ('B', 'C') --> ['A']  


#### Running Generic dataset with support 40% and confidence 42%

In [26]:
#Specify minimum support
minimum_support = 40
#Specify minimum confidence
minimum_confidence = 42

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_generic = main(generic,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Generic with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_generic,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules Generic with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_generic['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_generic['Antecedent'][i],rules_generic['Consequent'][i]))


 All association rules for Generic with support = 40 and Confidence = 42 

   Antecedent Consequent  Rule_Confidence
0           A          B             0.65
1           B          A             1.00
2           A          C             0.65
3           C          A             1.00
4           D          A             1.00
5           A          E             0.60
6           E          A             1.00
7           B          C             0.69
8           C          B             0.69
9      (A, B)        [C]             0.69
10     (A, C)        [B]             0.69
11     (B, C)        [A]             1.00 

---------------------------------------------------------------------

Final Association Rules Generic with support = 40 and Confidence = 42 

1) A --> B  
2) B --> A  
3) A --> C  
4) C --> A  
5) D --> A  
6) A --> E  
7) E --> A  
8) B --> C  
9) C --> B  
10) ('A', 'B') --> ['C']  
11) ('A', 'C') --> ['B']  
12) ('B', 'C') --> ['A']  


#### Running Generic dataset with support 10% and confidence 70%

In [27]:
#Specify minimum support
minimum_support = 10
#Specify minimum confidence
minimum_confidence = 70

#To determine association rules, the abover parameters(minimum support and confidence) are passed to main function alogn with the datset
for_rules,rules_generic = main(generic,minimum_support,minimum_confidence)

#Printing all Rules which satify mininmum confidence criterion. Rules: Tansactiona --> Transaction 
print('\n All association rules for Generic with support = {} and Confidence = {} \n' .format(minimum_support, minimum_confidence))
print(rules_generic,'\n')
print('---------------------------------------------------------------------\n')
#Printing Final association rules in format A,B --> C
print("Final Association Rules Generic with support = {} and Confidence = {} \n" .format(minimum_support, minimum_confidence))
for i in range(len(rules_generic['Rule_Confidence'])):
    print ("{}) {} --> {}  ".format(i+1, rules_generic['Antecedent'][i],rules_generic['Consequent'][i]))


 All association rules for Generic with support = 10 and Confidence = 70 

   Antecedent Consequent  Rule_Confidence
0           B          A             1.00
1           C          A             1.00
2           D          A             1.00
3           E          A             1.00
4           D          B             0.75
5           D          C             0.75
6      (B, C)        [A]             1.00
7      (A, D)        [B]             0.75
8      (B, D)        [A]             1.00
9      (B, E)        [A]             1.00
10     (A, D)        [C]             0.75
11     (C, D)        [A]             1.00
12     (C, E)        [A]             1.00
13     (D, E)        [A]             1.00
14     (B, D)        [C]             0.83
15     (C, D)        [B]             0.83
16     (D, E)        [B]             0.75
17  (A, B, D)        [C]             0.83
18  (A, C, D)        [B]             0.83
19  (B, C, D)        [A]             1.00
20  (B, C, E)        [A]             1.00
